# 기본세팅

In [1]:
### 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 설치 후 colab의 경우 Runtime > Restart session 필요

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Font directories:
	/root/.local/share/fonts
	/usr/local/share/fonts
	/usr/share/fonts
	/root/.fonts
	/usr/share/fonts/truetype
	/usr/share/fonts/truetype/dejavu
	/usr/share/fonts/truetype/liberation
	/usr/share/fonts/truetype/nanum
/root/.local/share/fonts: skipping, no such directory
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/dejavu: caching, new cache contents: 22 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/root/.fonts: skipping, no such directory
/usr/shar

In [32]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

In [33]:
import pandas as pd
import seaborn as sns
import numpy as np
import scipy
import scipy.stats as stats

# 간병인 플랫폼 어플 리뷰 분석
## 분석 목적 : 레어필드 간병인서비스와 유사한 사업을 운영하고 있는 경쟁업체 플랫폼의 어플 리뷰 분석
- 경쟁업체 분석을 통한 pain point 접근
- 환자용 어플과 간병인용 어플 분류하여 접근

### 분석 방향
1) 경쟁사 어플리케이션 리뷰 환자용/간병인용 분류
1) 경쟁사 어플리케이션 리뷰 긍/부정 분류
2) 긍정 리뷰 keyword 추출
3) 부정 리뷰 keyword 추출 (pain point)
4) 비교 차트 그리기 & 우리 사업의 방향성 제시

#### dataset 불러오기

In [34]:
# mongo에서 db 불러오는 함수
import pymongo
def dbconnect(collection) :
    client=pymongo.MongoClient('mongodb://trainings.iptime.org',45003)
    db= client['data_analysis']
    collection = db[collection]
    data = collection.find()
    df = pd.DataFrame(data)
    return df

In [35]:
# 중복값 제거하는 함수(contents를 기준으로)
def drop_duplicates(df, contents) : 
    df.drop_duplicates(subset=contents, keep='first', inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

In [36]:
# 특정 열에서 list를 str으로 변환하는 함수
def list_join(row) :
    row = ' '.join(row)
    return row

In [37]:
# 특정 열에서 \n 삭제하는 함수
def remove_text(row):
    if pd.notnull(row):
        row = row.replace(r'\n', ' ')
    return row

In [38]:
# 구글 플레이스토어 리뷰 불러오기
df_googlestore = dbconnect('caregive_platform_google')
# 애플 앱스토어 리뷰 불러오기
df_applestore = dbconnect('caregive_platform_apple')
# 행으로 데이터프레임 합치기
df_medical_platform = pd.concat([df_googlestore, df_applestore], axis=0)
df_medical_platform

,_id,app_name,user_score,user_date,user_comments,evaluation
0,66473d740f1921e26da151b3,케어닥,3,2024년 1월 24일,오늘 갑자기 에러가 자꾸 뜨네요 글구 보고 있던 창이 유지가 되지 않고 나갔다 들어...,3
1,66473d740f1921e26da151b4,케어닥,1,2024년 1월 19일,"업그레이드될수록 맘에 안듭니다!!간병비도 넘 싸고 지역별로 나누고 병원,재가도 나눠...",14
2,66473d740f1921e26da151b5,케어닥,5,2024년 1월 18일,간단하고 편리하며 실용적 입니다.,13
3,66473d740f1921e26da151b6,케어닥,5,2024년 4월 1일,입주경력많음,2
4,66473d740f1921e26da151b7,케어닥,5,2024년 1월 19일,알찬 정보를 제공해 주셔서 참으로 감사합니다~^^~,
...,...,...,...,...,...,...
167,664abf0b5d1ec9ef48becf1a,또하나의가족,5,2022-06-26T19:42:24-07:00,요양병원 정보가 필요했는데 앱으로 이렇게 간편하게 찾아볼 수 있어서 넘 좋네요,0
168,664abf0b5d1ec9ef48becf1b,또하나의가족,5,2022-06-15T18:11:44-07:00,앱으로 편하게 바로 상담 받을 수 있어 넘 편하고 조아요,0
169,664abf0b5d1ec9ef48becf1c,또하나의가족,5,2022-06-09T03:29:53-07:00,또가가 다른 사이트에 비해 정보도 많고 보기 편해서 \n\n요양원 찾을 때마다 이...,0
170,664abf0b5d1ec9ef48becf1d,또하나의가족,4,2022-06-08T01:37:25-07:00,입소 관련해서 상담 드렸는데 답변도 빠르고 친절했어요. 동네에 있는 시설들 다 찾아...,0


In [39]:
df_applestore['app_name'].value_counts()
# 제외할 겂 없음

app_name
케어닥       54
토닥씨       35
케어네이션     33
또하나의가족    30
병원잡       15
헬퍼         5
Name: count, dtype: int64

#### data 전처리

In [40]:
df_medical_platform.isnull().sum()

_id              0
app_name         0
user_score       0
user_date        0
user_comments    0
evaluation       0
dtype: int64

In [41]:
del df_medical_platform['evaluation']
df_medical_platform

,_id,app_name,user_score,user_date,user_comments
0,66473d740f1921e26da151b3,케어닥,3,2024년 1월 24일,오늘 갑자기 에러가 자꾸 뜨네요 글구 보고 있던 창이 유지가 되지 않고 나갔다 들어...
1,66473d740f1921e26da151b4,케어닥,1,2024년 1월 19일,"업그레이드될수록 맘에 안듭니다!!간병비도 넘 싸고 지역별로 나누고 병원,재가도 나눠..."
2,66473d740f1921e26da151b5,케어닥,5,2024년 1월 18일,간단하고 편리하며 실용적 입니다.
3,66473d740f1921e26da151b6,케어닥,5,2024년 4월 1일,입주경력많음
4,66473d740f1921e26da151b7,케어닥,5,2024년 1월 19일,알찬 정보를 제공해 주셔서 참으로 감사합니다~^^~
...,...,...,...,...,...
167,664abf0b5d1ec9ef48becf1a,또하나의가족,5,2022-06-26T19:42:24-07:00,요양병원 정보가 필요했는데 앱으로 이렇게 간편하게 찾아볼 수 있어서 넘 좋네요
168,664abf0b5d1ec9ef48becf1b,또하나의가족,5,2022-06-15T18:11:44-07:00,앱으로 편하게 바로 상담 받을 수 있어 넘 편하고 조아요
169,664abf0b5d1ec9ef48becf1c,또하나의가족,5,2022-06-09T03:29:53-07:00,또가가 다른 사이트에 비해 정보도 많고 보기 편해서 \n\n요양원 찾을 때마다 이...
170,664abf0b5d1ec9ef48becf1d,또하나의가족,4,2022-06-08T01:37:25-07:00,입소 관련해서 상담 드렸는데 답변도 빠르고 친절했어요. 동네에 있는 시설들 다 찾아...


#### 간병인용(지수) / 환자용(유경) 분리

In [42]:
df_medical_platform['app_name'].value_counts()

app_name
케어네이션                            1843
케어네이션(간병인용)                       928
좋은케어                              488
또하나의가족                            110
케어닥                               100
병원잡                                55
간병 나이팅게일                           42
토닥씨                                35
좋은 케어                              20
이웃하다                               17
케어파트너 요양보호사 장기요양 구직 취업 일자리 찾기      15
헬퍼                                 11
케어링크                               11
케어플러스                               9
헬로우케어                               6
캐어매치                                5
따뜻한돌봄                               4
Name: count, dtype: int64

In [43]:
# 환자용 
df_medical_platform_patient = df_medical_platform[df_medical_platform['app_name'].isin(['케어닥', '케어네이션', '헬퍼', '케어플러스', '이웃하다', '좋은 케어', '간병 나이팅게일', '또하나의가족', '헬로우케어', '토닥씨'])]
df_medical_platform_patient

,_id,app_name,user_score,user_date,user_comments
0,66473d740f1921e26da151b3,케어닥,3,2024년 1월 24일,오늘 갑자기 에러가 자꾸 뜨네요 글구 보고 있던 창이 유지가 되지 않고 나갔다 들어...
1,66473d740f1921e26da151b4,케어닥,1,2024년 1월 19일,"업그레이드될수록 맘에 안듭니다!!간병비도 넘 싸고 지역별로 나누고 병원,재가도 나눠..."
2,66473d740f1921e26da151b5,케어닥,5,2024년 1월 18일,간단하고 편리하며 실용적 입니다.
3,66473d740f1921e26da151b6,케어닥,5,2024년 4월 1일,입주경력많음
4,66473d740f1921e26da151b7,케어닥,5,2024년 1월 19일,알찬 정보를 제공해 주셔서 참으로 감사합니다~^^~
...,...,...,...,...,...
167,664abf0b5d1ec9ef48becf1a,또하나의가족,5,2022-06-26T19:42:24-07:00,요양병원 정보가 필요했는데 앱으로 이렇게 간편하게 찾아볼 수 있어서 넘 좋네요
168,664abf0b5d1ec9ef48becf1b,또하나의가족,5,2022-06-15T18:11:44-07:00,앱으로 편하게 바로 상담 받을 수 있어 넘 편하고 조아요
169,664abf0b5d1ec9ef48becf1c,또하나의가족,5,2022-06-09T03:29:53-07:00,또가가 다른 사이트에 비해 정보도 많고 보기 편해서 \n\n요양원 찾을 때마다 이...
170,664abf0b5d1ec9ef48becf1d,또하나의가족,4,2022-06-08T01:37:25-07:00,입소 관련해서 상담 드렸는데 답변도 빠르고 친절했어요. 동네에 있는 시설들 다 찾아...


In [44]:
# 간병인용 
df_medical_platform_caremanager = df_medical_platform[df_medical_platform['app_name'].isin(['케어닥', '케어네이션(간병인용)', '헬퍼', '케어링크', '케어매치', '케어파트너 요양보호사 장기요양 구직 취업 일자리 찾기', '병원잡'])]
df_medical_platform_caremanager

,_id,app_name,user_score,user_date,user_comments
0,66473d740f1921e26da151b3,케어닥,3,2024년 1월 24일,오늘 갑자기 에러가 자꾸 뜨네요 글구 보고 있던 창이 유지가 되지 않고 나갔다 들어...
1,66473d740f1921e26da151b4,케어닥,1,2024년 1월 19일,"업그레이드될수록 맘에 안듭니다!!간병비도 넘 싸고 지역별로 나누고 병원,재가도 나눠..."
2,66473d740f1921e26da151b5,케어닥,5,2024년 1월 18일,간단하고 편리하며 실용적 입니다.
3,66473d740f1921e26da151b6,케어닥,5,2024년 4월 1일,입주경력많음
4,66473d740f1921e26da151b7,케어닥,5,2024년 1월 19일,알찬 정보를 제공해 주셔서 참으로 감사합니다~^^~
...,...,...,...,...,...
137,664abf095d1ec9ef48becefc,병원잡,3,2020-12-02T19:22:26-07:00,렉도 없고 항상 공고가 올라와서 너무 좋은 것 같아요!
138,664abf095d1ec9ef48becefd,병원잡,5,2020-11-27T04:03:45-07:00,이곳저곳 알아 보기 힘들 때 병원잡 어플 하나면\n한눈에 다 알아보고 일자리 구할 ...
139,664abf095d1ec9ef48becefe,병원잡,5,2020-11-07T23:03:57-07:00,한눈에 빠르게 전국 병원의 구인을 볼수 있어서 좋네요\n이력서 등록해 두면 편리 합니다
140,664abf095d1ec9ef48beceff,병원잡,4,2020-11-04T22:19:43-07:00,공고 한눈에 보기 좋아서 편리해요


##### 1. 경쟁사 어플리케이션 리뷰 긍부정 분석

##### 플랫폼별 분류

In [45]:
# 

In [46]:
df_medical_platform_caremanager.drop(columns='_id', inplace=True)

/tmp/ipykernel_28782/1921062872.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_medical_platform_caremanager.drop(columns='_id', inplace=True)


In [47]:
def datetime_change(row):
    try:
        row = pd.to_datetime(row)
    except :
        row = pd.to_datetime(row, format='%Y년 %m월 %d일')
    return row


In [48]:
df_medical_platform_caremanager['user_date_datetime'] = df_medical_platform_caremanager['user_date'].apply(datetime_change)

/tmp/ipykernel_28782/1184814773.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_medical_platform_caremanager['user_date_datetime'] = df_medical_platform_caremanager['user_date'].apply(datetime_change)


In [49]:
# 'user_date' 열을 datetime 형식으로 변환
df_medical_platform_caremanager['user_date_datetime'] = pd.to_datetime(df_medical_platform_caremanager['user_date_datetime'], errors='coerce', format='%Y년 %m월 %d일')

# 년, 월, 일 정보 추출
df_medical_platform_caremanager['year'] = df_medical_platform_caremanager['user_date_datetime'].dt.year
df_medical_platform_caremanager['month'] = df_medical_platform_caremanager['user_date_datetime'].dt.month
df_medical_platform_caremanager['day'] = df_medical_platform_caremanager['user_date_datetime'].dt.day

# 결과 확인
print(df_medical_platform_caremanager[['user_date_datetime', 'year', 'month', 'day']])

    user_date_datetime    year  month   day
0           2024-01-24  2024.0    1.0  24.0
1           2024-01-19  2024.0    1.0  19.0
2           2024-01-18  2024.0    1.0  18.0
3           2024-04-01  2024.0    4.0   1.0
4           2024-01-19  2024.0    1.0  19.0
..                 ...     ...    ...   ...
137                NaT     NaN    NaN   NaN
138                NaT     NaN    NaN   NaN
139                NaT     NaN    NaN   NaN
140                NaT     NaN    NaN   NaN
141                NaT     NaN    NaN   NaN

[1120 rows x 4 columns]


/tmp/ipykernel_28782/3458398433.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_medical_platform_caremanager['user_date_datetime'] = pd.to_datetime(df_medical_platform_caremanager['user_date_datetime'], errors='coerce', format='%Y년 %m월 %d일')
/tmp/ipykernel_28782/3458398433.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_medical_platform_caremanager['year'] = df_medical_platform_caremanager['user_date_datetime'].dt.year
/tmp/ipykernel_28782/3458398433.py:6: SettingWithCopyWarning: 
A value is t

##### 형태소 분석기

###### mecab

In [56]:
# 숫자 삭제
df_medical_platform_caremanager['review_preprocess'] = df_medical_platform_caremanager['user_comments'].str.replace('[0-9]','',regex=True)

/tmp/ipykernel_28782/4246750757.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_medical_platform_caremanager['review_preprocess'] = df_medical_platform_caremanager['user_comments'].str.replace('[0-9]','',regex=True)


In [51]:
!pip install python-mecab-ko

In [53]:
from mecab import MeCab
mecab = MeCab()

In [55]:
mecab.pos(str(df_medical_platform_caremanager['user_comments'][4]))

[('4', 'SN'),
 ('알찬', 'NNG'),
 ('정보', 'NNG'),
 ('를', 'JKO'),
 ('제공', 'NNG'),
 ('해', 'XSV+EC'),
 ('주', 'VX'),
 ('셔서', 'EP+EC'),
 ('참으로', 'MAG'),
 ('감사', 'NNG'),
 ('합니다', 'XSV+EF'),
 ('~^^~', 'SY'),
 ('4', 'SN'),
 ('처음', 'NNG'),
 ('에', 'JKB'),
 ('간병', 'NNG'),
 ('서비스', 'NNG'),
 ('신청', 'NNG'),
 ('하', 'VV'),
 ('고', 'EC'),
 ('나', 'VX'),
 ('서', 'EC'),
 ('\\', 'SY'),
 ('n', 'SL'),
 ('나중', 'NNG'),
 ('에', 'JKB'),
 ('신청서', 'NNG'),
 ('수정', 'NNG'),
 ('>', 'SY'),
 ('맨', 'MM'),
 ('하단', 'NNG'),
 ('에', 'JKB'),
 ('특이', 'NNG'),
 ('사항', 'NNG'),
 ('적', 'XSN'),
 ('.', 'SF'),
 ('..', 'SY'),
 ('Name', 'SL'),
 (':', 'SC'),
 ('user', 'SL'),
 ('_', 'SY'),
 ('comments', 'SL'),
 (',', 'SC'),
 ('dtype', 'SL'),
 (':', 'SC'),
 ('object', 'SL')]

In [54]:
mecab.pos(df_medical_platform_caremanager['user_comments'][4])

TypeError: set_sentence(): incompatible function arguments. The following argument types are supported:
    1. (self: _mecab.Lattice, arg0: str) -> None
    2. (self: _mecab.Lattice, arg0: str, arg1: int) -> None

Invoked with: <repr raised Error>, 4                         알찬 정보를 제공해 주셔서 참으로 감사합니다~^^~
4    처음에 간병 서비스 신청 하고 나서\n나중에 신청서 수정 > 맨 하단에 특이사항 적...
Name: user_comments, dtype: object

In [92]:
def mecab_tag(text):
    morphs_nouns = []
    morphs_nouns_list = []
    for sentence in text:
        # 명사, 형용사, 동사
        morphs = [morph for morph, tag in mecab.pos(sentence) if tag in ['VV+ETN', 'MM', 'NNG', 'NNP', 'NP', 'NR', 'VA', 'VV']]
        morphs_nouns.append(' '.join(morphs))
    
    return morphs_nouns

In [93]:
df_appreview.isnull().sum()

_id                  0
app_name             0
user_score           0
user_date            0
user_comments        0
review_preprocess    0
dtype: int64

In [94]:
morphs_nouns = mecab_tag(df_appreview['user_comments'])

In [95]:
df_appreview['review_clean_mecab'] = morphs_nouns

In [96]:
df_appreview

,_id,app_name,user_score,user_date,user_comments,review_preprocess,review_clean_mecab
0,662f0b3073f7ce16925e7c72,triple,5,2024년 4월 20일,"트리플 덕분에 최근에 다녀온 여행 계획을 편하게 짰습니다. 다만, 아쉬운 점이 있습...","트리플 덕분에 최근에 다녀온 여행 계획을 편하게 짰습니다. 다만, 아쉬운 점이 있습...",트리플 덕분 최근 여행 계획 편하 점 있 가계부 지출 항목 다양 좋 교통비 예 들 ...
1,662f0b3073f7ce16925e7c73,triple,4,2024년 4월 2일,여행 일정 잡을 때. 가장 애용하고 손이 많이 가는 앱입니다 :) 너무너무 아쉬운 ...,여행 일정 잡을 때. 가장 애용하고 손이 많이 가는 앱입니다 :) 너무너무 아쉬운 ...,여행 일정 잡 때 애용 손 가 앱 점 버전 없 여행 일정 결국 마지막 각 잡 때 많...
2,662f0b3073f7ce16925e7c74,triple,5,2024년 3월 31일,"갤럭시 폴드하고 태블릿에서 더 좋은 사용성을 가진 UX가 제공되서 좋네요 다만, P...","갤럭시 폴드하고 태블릿에서 더 좋은 사용성을 가진 UX가 제공되서 좋네요 다만, P...",갤럭시 폴드 태블릿 좋 사용 제공 좋 버전 오프라인 모드 지원 없 같 계획 일정 때...
3,662f0b3073f7ce16925e7c75,triple,5,2024년 3월 3일,"진짜 편함. 특히 친구들이랑 같이 여행할 때 계획 짜고, 가계부 적어서 자동으로 1...","진짜 편함. 특히 친구들이랑 같이 여행할 때 계획 짜고, 가계부 적어서 자동으로 /...",친구 여행 때 계획 짜 가계부 적 자동 하 날씨 환율 확인 되 최고 하나 점 오프라...
4,662f0b3073f7ce16925e7c76,triple,5,2024년 3월 4일,같이 일정을 짜고 공유할 수 있다는 게 좋고 무엇보다 여기저기 영상 뒤지고 인스타 ...,같이 일정을 짜고 공유할 수 있다는 게 좋고 무엇보다 여기저기 영상 뒤지고 인스타 ...,일정 짜 공유 있 좋 무엇 여기저기 영상 뒤지 인 스타 피드 뒤지 하 대표 관광지 ...
...,...,...,...,...,...,...,...
3896,6645de8483f867cdd7e8996a,대한민국 구석구석,5,2015년 9월 21일,ㅡ굿,ㅡ굿,
3897,6645de8483f867cdd7e8996b,대한민국 구석구석,5,2015년 2월 25일,박서규,박서규,박 서규
3898,6645de8583f867cdd7e8996c,대한민국 구석구석,5,2013년 6월 29일,최곡,최곡,곡
3899,6645de8583f867cdd7e8996d,대한민국 구석구석,5,2014년 12월 7일,영호,영호,영호


In [97]:
df_appreview.query('review_clean_mecab == ""')

,_id,app_name,user_score,user_date,user_comments,review_preprocess,review_clean_mecab
2001,662f183961d9aa4564c079ef,triple,5,2023년 1월 2일,아직까지는,아직까지는,
2009,662f183961d9aa4564c079f7,triple,5,2023년 8월 28일,아주 굳이에요,아주 굳이에요,
2014,662f183961d9aa4564c079fc,triple,5,2022년 1월 30일,넘편해요,넘편해요,
2019,662f183a61d9aa4564c07a01,triple,5,2019년 7월 17일,good,good,
2020,662f183a61d9aa4564c07a02,triple,5,2019년 7월 4일,good,good,
2022,662f183a61d9aa4564c07a04,triple,5,2019년 5월 31일,완전편함,완전편함,
2023,662f183a61d9aa4564c07a05,triple,5,2019년 4월 12일,good,good,
2029,662f183a61d9aa4564c07a0b,triple,5,2022년 4월 21일,써보세요!!,써보세요!!,
2034,662f183a61d9aa4564c07a10,triple,5,2019년 5월 6일,so good!!!,so good!!!,
2035,662f183b61d9aa4564c07a11,triple,5,2021년 11월 1일,Perfect,Perfect,


###### okt

In [98]:
!pip install konlpy

In [99]:
from konlpy.tag import Okt
okt = Okt()

In [143]:
# 조사, 어미, 구두점 같은 것을 제거
def okt_clean(text):
    clean_text = []
    for word in okt.pos(text, stem=True):
        if word[1] not in ['Josa', 'Eomi', 'Punctuation', 'Adverb', 'Suffix', 'KoreanParticle', 'Alpha', 'Verb']:
            clean_text.append(word[0])

    return " ".join(clean_text)

In [144]:
df_appreview['review_clean_okt'] = df_appreview['review_preprocess'].apply(okt_clean)

In [145]:
df_appreview['review_clean_okt'][11260:11270]

Series([], Name: review_clean_okt, dtype: object)

In [146]:
okt.pos(df_appreview['review_clean_okt'][10])

[('여행', 'Noun'),
 ('사용', 'Noun'),
 ('있다', 'Adjective'),
 ('다만', 'Noun'),
 ('인터페이스', 'Noun'),
 ('직관', 'Noun'),
 ('이용', 'Noun'),
 ('불편하다', 'Adjective'),
 ('점', 'Noun'),
 ('있다', 'Adjective'),
 ('가장', 'Noun'),
 ('부분', 'Noun'),
 ('여행', 'Noun'),
 ('일정', 'Noun'),
 ('수정', 'Noun'),
 ('탭', 'Noun'),
 ('때', 'Noun'),
 ('날짜', 'Noun'),
 ('예시', 'Noun'),
 ('를', 'Noun'),
 ('선택', 'Noun'),
 ('점', 'Noun'),
 ('이다', 'Josa'),
 ('그', 'Noun'),
 ('부분', 'Noun'),
 ('밑줄', 'Noun'),
 ('하지만', 'Conjunction'),
 ('눈', 'Noun'),
 ('어디', 'Noun'),
 ('그', 'Noun'),
 ('화면', 'Noun'),
 ('수', 'Noun'),
 ('있다', 'Adjective'),
 ('알', 'Noun'),
 ('수', 'Noun'),
 ('없다', 'Adjective'),
 ('처음', 'Noun'),
 ('애', 'Noun'),
 ('이', 'Noun'),
 ('부분', 'Noun'),
 ('정말', 'Noun'),
 ('좋다', 'Adjective'),
 ('것', 'Noun'),
 ('같다', 'Adjective')]

In [147]:
# 문장 정리 및 재정제
def solve_text(text):
    change_dict = {
        '바 우 처': '바우처'
        , '고객 센터' : '고객센터'
        , '단 언 컨데' : '단언컨데'
        , '별 점' : '별점'
        , '성 공률' : '성공률'
        , '페북' : '페이스북'
        , '트립어드 바이 저' : '트립어드바이저'
        , '사용자 들' : '사용자들'
        , '사람 들' : '사람들'
        , '믿음 직' : '믿음직'
        , '동반 자' : '동반자'
        , '일반 적' : '일반적'
        , '직관 적' : '직관적'
        , '마이 리얼 트립' : '마이리얼트립'
        , '엑 티비 티' : '엑티비티'
        , '결재' : '결제'
        , '대 구' : '대구'
        , '자유 여 행' : '자유여행'
        , '할인 율' : '할인율'
        , '필수 픔' : '필수품'
        , '현 지' : '현지'
        , '예 매' : '예매'
        , '항공 료' : '항공료'
        , '여 행시' : '여행시'
        , '한 달동 안' : '한달동안'
        , '카카오 톡' : '카카오톡'
        , '쵝오' : '최고'
        , '낚시 성' : '낚시성'
        , '투 닥투닥' : '투닥투닥'
        , '필터 링' : '필터링'
        , '자체 적' : '자체적'
        , '동 일' : '동일'
        , '자다' : '잘'
        , '용하다' : '이용하다'
        , '어 울다' : '어울리다'
        , '예 약' : '예약'
        , '애러' : '에러'
    }
    for i in change_dict:
        text = text.replace(i,change_dict[i])
    return text

In [148]:
df_appreview

,_id,app_name,user_score,user_date,user_comments,review_preprocess,review_clean_mecab,review_clean_okt
0,662f0b3073f7ce16925e7c72,triple,5,2024년 4월 20일,"트리플 덕분에 최근에 다녀온 여행 계획을 편하게 짰습니다. 다만, 아쉬운 점이 있습...","트리플 덕분에 최근에 다녀온 여행 계획을 편하게 짰습니다. 다만, 아쉬운 점이 있습...",덕분 최근 여행 계획 편하 점 있 가계부 지출 항목 다양 좋 교통비 예 들 항공 ...,트리플 덕분 최근 여행 계획 편하다 다만 아쉽다 점 있다 가계부 지 출항 목 좀 더...
1,662f0b3073f7ce16925e7c73,triple,4,2024년 4월 2일,여행 일정 잡을 때. 가장 애용하고 손이 많이 가는 앱입니다 :) 너무너무 아쉬운 ...,여행 일정 잡을 때. 가장 애용하고 손이 많이 가는 앱입니다 :) 너무너무 아쉬운 ...,여행 일정 잡 때 애용 손 가 앱 점 버전 없 여행 일정 결국 마지막 각 잡 때 많...,여행 일정 잡 때 가장 애용 손 앱 이다 아쉽다 점 버전 없다 게 여행 일정 마지막...
2,662f0b3073f7ce16925e7c74,triple,5,2024년 3월 31일,"갤럭시 폴드하고 태블릿에서 더 좋은 사용성을 가진 UX가 제공되서 좋네요 다만, P...","갤럭시 폴드하고 태블릿에서 더 좋은 사용성을 가진 UX가 제공되서 좋네요 다만, P...",갤럭시 폴드 태블릿 좋 사용 제공 좋 버전 오프라인 모드 지원 없 같 계획 일정 때...,갤럭시 폴 드하 태블릿 더 좋다 사용성 제공 좋다 다만 버전 오프라인 모드 지원 주...
3,662f0b3073f7ce16925e7c75,triple,5,2024년 3월 3일,"진짜 편함. 특히 친구들이랑 같이 여행할 때 계획 짜고, 가계부 적어서 자동으로 1...","진짜 편함. 특히 친구들이랑 같이 여행할 때 계획 짜고, 가계부 적어서 자동으로 /...",친구 여행 때 계획 짜 가계부 적 자동 하 날씨 환율 확인 되 최고 하나 점 오프라...,진짜 편하다 친구 여행 때 계획 가계부 자동 날씨 환율 확인 바로 거 최고 하나 아...
4,662f0b3073f7ce16925e7c76,triple,5,2024년 3월 4일,같이 일정을 짜고 공유할 수 있다는 게 좋고 무엇보다 여기저기 영상 뒤지고 인스타 ...,같이 일정을 짜고 공유할 수 있다는 게 좋고 무엇보다 여기저기 영상 뒤지고 인스타 ...,일정 짜 공유 있 좋 무엇 여기저기 영상 뒤지 인 스타 피드 뒤지 하 대표 관광지 ...,일정 공유 수 있다 게 좋다 무엇 여기저기 영상 인스타 피드 대표 관광지 대표 스팟...
...,...,...,...,...,...,...,...,...
3896,6645de8483f867cdd7e8996a,대한민국 구석구석,5,2015년 9월 21일,ㅡ굿,ㅡ굿,,굿
3897,6645de8483f867cdd7e8996b,대한민국 구석구석,5,2015년 2월 25일,박서규,박서규,박 서규,박서 규
3898,6645de8583f867cdd7e8996c,대한민국 구석구석,5,2013년 6월 29일,최곡,최곡,곡,최곡
3899,6645de8583f867cdd7e8996d,대한민국 구석구석,5,2014년 12월 7일,영호,영호,영호,영호


In [149]:
df_appreview.query('review_clean_okt == ""')

,_id,app_name,user_score,user_date,user_comments,review_preprocess,review_clean_mecab,review_clean_okt
2001,662f183961d9aa4564c079ef,triple,5,2023년 1월 2일,아직까지는,아직까지는,,
2019,662f183a61d9aa4564c07a01,triple,5,2019년 7월 17일,good,good,,
2020,662f183a61d9aa4564c07a02,triple,5,2019년 7월 4일,good,good,,
2023,662f183a61d9aa4564c07a05,triple,5,2019년 4월 12일,good,good,,
2029,662f183a61d9aa4564c07a0b,triple,5,2022년 4월 21일,써보세요!!,써보세요!!,,
2034,662f183a61d9aa4564c07a10,triple,5,2019년 5월 6일,so good!!!,so good!!!,,
2035,662f183b61d9aa4564c07a11,triple,5,2021년 11월 1일,Perfect,Perfect,,
2379,662f3b6134754d6a96872776,triple,5,2021-11-15T01:54:57-07:00,ㅇ,ㅇ,,
2447,662f3b6234754d6a968727ba,triple,5,2020-10-26T20:19:31-07:00,very very,very very,,
2482,662f3b6234754d6a968727dd,triple,5,2020-07-28T07:07:11-07:00,.,.,,


In [222]:
# 어플명 제거
import re

pattern = '바로|로딩|언제|그리고|하나|미치다|와이파이|다른|부탁드리다|지금|자꾸|넥서스|튕깁니|그렇다|서울|고맙다|감사|고생|완전|무조건|덕분|진짜|스카이스캐너|트립어드바이저|트리플|마이리얼트립|스카이 스캐너|마이 리얼 트립|트립 어드 바이저|에어부산|되다|하다|이다|같다|보다|돼다|니당다|네이버|로그인|설치|있다|없다|그냥|별로|업데이트|오류|어플|아니다|확인|다시|몇번|제대로|설치|화면|실행|보통|개선|기능|하루|추가|유용|어플|정말|최고|짱짱|굿굿|좋다|여행|아쉽다|항상|아주|쓰기|자주|개발자|내내|여행자|대한민국|구석구석|대한민국 구석구석|국내|사용|계속|이렇다|다운'

df_appreview['review_clean_mecab'] = df_appreview['review_clean_mecab'].str.replace(pattern, '', regex=True)
df_appreview['review_clean_okt'] = df_appreview['review_clean_okt'].str.replace(pattern, '', regex=True)

In [223]:
df_appreview

,_id,app_name,user_score,user_date,user_comments,review_preprocess,review_clean_mecab,review_clean_okt
0,662f0b3073f7ce16925e7c72,triple,5,2024년 4월 20일,"트리플 덕분에 최근에 다녀온 여행 계획을 편하게 짰습니다. 다만, 아쉬운 점이 있습...","트리플 덕분에 최근에 다녀온 여행 계획을 편하게 짰습니다. 다만, 아쉬운 점이 있습...",최근 계획 편하 점 있 가계부 지출 항목 다양 좋 교통비 예 들 항공 교통 두...,최근 계획 편 다만 점 가계부 지 출항 목 좀 더 다양 교통비 예 항공 ...
1,662f0b3073f7ce16925e7c73,triple,4,2024년 4월 2일,여행 일정 잡을 때. 가장 애용하고 손이 많이 가는 앱입니다 :) 너무너무 아쉬운 ...,여행 일정 잡을 때. 가장 애용하고 손이 많이 가는 앱입니다 :) 너무너무 아쉬운 ...,일정 잡 때 애용 손 가 앱 점 버전 없 일정 결국 마지막 각 잡 때 많 쓰 ...,일정 잡 때 가장 애용 손 앱 점 버전 게 일정 마지막 각 잡고 로 때 많...
2,662f0b3073f7ce16925e7c74,triple,5,2024년 3월 31일,"갤럭시 폴드하고 태블릿에서 더 좋은 사용성을 가진 UX가 제공되서 좋네요 다만, P...","갤럭시 폴드하고 태블릿에서 더 좋은 사용성을 가진 UX가 제공되서 좋네요 다만, P...",갤럭시 폴드 태블릿 좋 제공 좋 버전 오프라인 모드 지원 없 같 계획 일정 때 모...,갤럭시 폴 드하 태블릿 더 성 제공 다만 버전 오프라인 모드 지원 주시 더 나위...
3,662f0b3073f7ce16925e7c75,triple,5,2024년 3월 3일,"진짜 편함. 특히 친구들이랑 같이 여행할 때 계획 짜고, 가계부 적어서 자동으로 1...","진짜 편함. 특히 친구들이랑 같이 여행할 때 계획 짜고, 가계부 적어서 자동으로 /...",친구 때 계획 짜 가계부 적 자동 하 날씨 환율 되 점 오프라인 일정 있 좋,편 친구 때 계획 가계부 자동 날씨 환율 거 점 오프라인 일정 볼 수
4,662f0b3073f7ce16925e7c76,triple,5,2024년 3월 4일,같이 일정을 짜고 공유할 수 있다는 게 좋고 무엇보다 여기저기 영상 뒤지고 인스타 ...,같이 일정을 짜고 공유할 수 있다는 게 좋고 무엇보다 여기저기 영상 뒤지고 인스타 ...,일정 짜 공유 있 좋 무엇 여기저기 영상 뒤지 인 스타 피드 뒤지 하 대표 관광지 ...,일정 공유 수 게 무엇 여기저기 영상 인스타 피드 대표 관광지 대표 스팟 정리 ...
...,...,...,...,...,...,...,...,...
3896,6645de8483f867cdd7e8996a,대한민국 구석구석,5,2015년 9월 21일,ㅡ굿,ㅡ굿,,굿
3897,6645de8483f867cdd7e8996b,대한민국 구석구석,5,2015년 2월 25일,박서규,박서규,박 서규,박서 규
3898,6645de8583f867cdd7e8996c,대한민국 구석구석,5,2013년 6월 29일,최곡,최곡,곡,최곡
3899,6645de8583f867cdd7e8996d,대한민국 구석구석,5,2014년 12월 7일,영호,영호,영호,영호


###### okt로 선택

In [224]:
df_appreview

,_id,app_name,user_score,user_date,user_comments,review_preprocess,review_clean_mecab,review_clean_okt
0,662f0b3073f7ce16925e7c72,triple,5,2024년 4월 20일,"트리플 덕분에 최근에 다녀온 여행 계획을 편하게 짰습니다. 다만, 아쉬운 점이 있습...","트리플 덕분에 최근에 다녀온 여행 계획을 편하게 짰습니다. 다만, 아쉬운 점이 있습...",최근 계획 편하 점 있 가계부 지출 항목 다양 좋 교통비 예 들 항공 교통 두...,최근 계획 편 다만 점 가계부 지 출항 목 좀 더 다양 교통비 예 항공 ...
1,662f0b3073f7ce16925e7c73,triple,4,2024년 4월 2일,여행 일정 잡을 때. 가장 애용하고 손이 많이 가는 앱입니다 :) 너무너무 아쉬운 ...,여행 일정 잡을 때. 가장 애용하고 손이 많이 가는 앱입니다 :) 너무너무 아쉬운 ...,일정 잡 때 애용 손 가 앱 점 버전 없 일정 결국 마지막 각 잡 때 많 쓰 ...,일정 잡 때 가장 애용 손 앱 점 버전 게 일정 마지막 각 잡고 로 때 많...
2,662f0b3073f7ce16925e7c74,triple,5,2024년 3월 31일,"갤럭시 폴드하고 태블릿에서 더 좋은 사용성을 가진 UX가 제공되서 좋네요 다만, P...","갤럭시 폴드하고 태블릿에서 더 좋은 사용성을 가진 UX가 제공되서 좋네요 다만, P...",갤럭시 폴드 태블릿 좋 제공 좋 버전 오프라인 모드 지원 없 같 계획 일정 때 모...,갤럭시 폴 드하 태블릿 더 성 제공 다만 버전 오프라인 모드 지원 주시 더 나위...
3,662f0b3073f7ce16925e7c75,triple,5,2024년 3월 3일,"진짜 편함. 특히 친구들이랑 같이 여행할 때 계획 짜고, 가계부 적어서 자동으로 1...","진짜 편함. 특히 친구들이랑 같이 여행할 때 계획 짜고, 가계부 적어서 자동으로 /...",친구 때 계획 짜 가계부 적 자동 하 날씨 환율 되 점 오프라인 일정 있 좋,편 친구 때 계획 가계부 자동 날씨 환율 거 점 오프라인 일정 볼 수
4,662f0b3073f7ce16925e7c76,triple,5,2024년 3월 4일,같이 일정을 짜고 공유할 수 있다는 게 좋고 무엇보다 여기저기 영상 뒤지고 인스타 ...,같이 일정을 짜고 공유할 수 있다는 게 좋고 무엇보다 여기저기 영상 뒤지고 인스타 ...,일정 짜 공유 있 좋 무엇 여기저기 영상 뒤지 인 스타 피드 뒤지 하 대표 관광지 ...,일정 공유 수 게 무엇 여기저기 영상 인스타 피드 대표 관광지 대표 스팟 정리 ...
...,...,...,...,...,...,...,...,...
3896,6645de8483f867cdd7e8996a,대한민국 구석구석,5,2015년 9월 21일,ㅡ굿,ㅡ굿,,굿
3897,6645de8483f867cdd7e8996b,대한민국 구석구석,5,2015년 2월 25일,박서규,박서규,박 서규,박서 규
3898,6645de8583f867cdd7e8996c,대한민국 구석구석,5,2013년 6월 29일,최곡,최곡,곡,최곡
3899,6645de8583f867cdd7e8996d,대한민국 구석구석,5,2014년 12월 7일,영호,영호,영호,영호


In [225]:
df_appreview.isin(['']).sum()

_id                     0
app_name                0
user_score              0
user_date               0
user_comments           0
review_preprocess       0
review_clean_mecab     72
review_clean_okt      139
dtype: int64

In [226]:
df_appreview_okt = df_appreview.copy()

In [227]:
df_appreview_okt.replace({'': np.nan}, inplace=True)

In [228]:
df_appreview_okt.dropna(subset=['review_clean_okt'], inplace=True)

In [229]:
df_appreview_okt.isin(['']).sum()

_id                   0
app_name              0
user_score            0
user_date             0
user_comments         0
review_preprocess     0
review_clean_mecab    0
review_clean_okt      0
dtype: int64

In [230]:
df_appreview_okt = df_appreview_okt.reset_index()

In [231]:
df_appreview_okt

,index,_id,app_name,user_score,user_date,user_comments,review_preprocess,review_clean_mecab,review_clean_okt
0,0,662f0b3073f7ce16925e7c72,triple,5,2024년 4월 20일,"트리플 덕분에 최근에 다녀온 여행 계획을 편하게 짰습니다. 다만, 아쉬운 점이 있습...","트리플 덕분에 최근에 다녀온 여행 계획을 편하게 짰습니다. 다만, 아쉬운 점이 있습...",최근 계획 편하 점 있 가계부 지출 항목 다양 좋 교통비 예 들 항공 교통 두...,최근 계획 편 다만 점 가계부 지 출항 목 좀 더 다양 교통비 예 항공 ...
1,1,662f0b3073f7ce16925e7c73,triple,4,2024년 4월 2일,여행 일정 잡을 때. 가장 애용하고 손이 많이 가는 앱입니다 :) 너무너무 아쉬운 ...,여행 일정 잡을 때. 가장 애용하고 손이 많이 가는 앱입니다 :) 너무너무 아쉬운 ...,일정 잡 때 애용 손 가 앱 점 버전 없 일정 결국 마지막 각 잡 때 많 쓰 ...,일정 잡 때 가장 애용 손 앱 점 버전 게 일정 마지막 각 잡고 로 때 많...
2,2,662f0b3073f7ce16925e7c74,triple,5,2024년 3월 31일,"갤럭시 폴드하고 태블릿에서 더 좋은 사용성을 가진 UX가 제공되서 좋네요 다만, P...","갤럭시 폴드하고 태블릿에서 더 좋은 사용성을 가진 UX가 제공되서 좋네요 다만, P...",갤럭시 폴드 태블릿 좋 제공 좋 버전 오프라인 모드 지원 없 같 계획 일정 때 모...,갤럭시 폴 드하 태블릿 더 성 제공 다만 버전 오프라인 모드 지원 주시 더 나위...
3,3,662f0b3073f7ce16925e7c75,triple,5,2024년 3월 3일,"진짜 편함. 특히 친구들이랑 같이 여행할 때 계획 짜고, 가계부 적어서 자동으로 1...","진짜 편함. 특히 친구들이랑 같이 여행할 때 계획 짜고, 가계부 적어서 자동으로 /...",친구 때 계획 짜 가계부 적 자동 하 날씨 환율 되 점 오프라인 일정 있 좋,편 친구 때 계획 가계부 자동 날씨 환율 거 점 오프라인 일정 볼 수
4,4,662f0b3073f7ce16925e7c76,triple,5,2024년 3월 4일,같이 일정을 짜고 공유할 수 있다는 게 좋고 무엇보다 여기저기 영상 뒤지고 인스타 ...,같이 일정을 짜고 공유할 수 있다는 게 좋고 무엇보다 여기저기 영상 뒤지고 인스타 ...,일정 짜 공유 있 좋 무엇 여기저기 영상 뒤지 인 스타 피드 뒤지 하 대표 관광지 ...,일정 공유 수 게 무엇 여기저기 영상 인스타 피드 대표 관광지 대표 스팟 정리 ...
...,...,...,...,...,...,...,...,...,...
3757,3896,6645de8483f867cdd7e8996a,대한민국 구석구석,5,2015년 9월 21일,ㅡ굿,ㅡ굿,NaN,굿
3758,3897,6645de8483f867cdd7e8996b,대한민국 구석구석,5,2015년 2월 25일,박서규,박서규,박 서규,박서 규
3759,3898,6645de8583f867cdd7e8996c,대한민국 구석구석,5,2013년 6월 29일,최곡,최곡,곡,최곡
3760,3899,6645de8583f867cdd7e8996d,대한민국 구석구석,5,2014년 12월 7일,영호,영호,영호,영호


##### 긍부정 분석

In [232]:
df_appreview_okt

,index,_id,app_name,user_score,user_date,user_comments,review_preprocess,review_clean_mecab,review_clean_okt
0,0,662f0b3073f7ce16925e7c72,triple,5,2024년 4월 20일,"트리플 덕분에 최근에 다녀온 여행 계획을 편하게 짰습니다. 다만, 아쉬운 점이 있습...","트리플 덕분에 최근에 다녀온 여행 계획을 편하게 짰습니다. 다만, 아쉬운 점이 있습...",최근 계획 편하 점 있 가계부 지출 항목 다양 좋 교통비 예 들 항공 교통 두...,최근 계획 편 다만 점 가계부 지 출항 목 좀 더 다양 교통비 예 항공 ...
1,1,662f0b3073f7ce16925e7c73,triple,4,2024년 4월 2일,여행 일정 잡을 때. 가장 애용하고 손이 많이 가는 앱입니다 :) 너무너무 아쉬운 ...,여행 일정 잡을 때. 가장 애용하고 손이 많이 가는 앱입니다 :) 너무너무 아쉬운 ...,일정 잡 때 애용 손 가 앱 점 버전 없 일정 결국 마지막 각 잡 때 많 쓰 ...,일정 잡 때 가장 애용 손 앱 점 버전 게 일정 마지막 각 잡고 로 때 많...
2,2,662f0b3073f7ce16925e7c74,triple,5,2024년 3월 31일,"갤럭시 폴드하고 태블릿에서 더 좋은 사용성을 가진 UX가 제공되서 좋네요 다만, P...","갤럭시 폴드하고 태블릿에서 더 좋은 사용성을 가진 UX가 제공되서 좋네요 다만, P...",갤럭시 폴드 태블릿 좋 제공 좋 버전 오프라인 모드 지원 없 같 계획 일정 때 모...,갤럭시 폴 드하 태블릿 더 성 제공 다만 버전 오프라인 모드 지원 주시 더 나위...
3,3,662f0b3073f7ce16925e7c75,triple,5,2024년 3월 3일,"진짜 편함. 특히 친구들이랑 같이 여행할 때 계획 짜고, 가계부 적어서 자동으로 1...","진짜 편함. 특히 친구들이랑 같이 여행할 때 계획 짜고, 가계부 적어서 자동으로 /...",친구 때 계획 짜 가계부 적 자동 하 날씨 환율 되 점 오프라인 일정 있 좋,편 친구 때 계획 가계부 자동 날씨 환율 거 점 오프라인 일정 볼 수
4,4,662f0b3073f7ce16925e7c76,triple,5,2024년 3월 4일,같이 일정을 짜고 공유할 수 있다는 게 좋고 무엇보다 여기저기 영상 뒤지고 인스타 ...,같이 일정을 짜고 공유할 수 있다는 게 좋고 무엇보다 여기저기 영상 뒤지고 인스타 ...,일정 짜 공유 있 좋 무엇 여기저기 영상 뒤지 인 스타 피드 뒤지 하 대표 관광지 ...,일정 공유 수 게 무엇 여기저기 영상 인스타 피드 대표 관광지 대표 스팟 정리 ...
...,...,...,...,...,...,...,...,...,...
3757,3896,6645de8483f867cdd7e8996a,대한민국 구석구석,5,2015년 9월 21일,ㅡ굿,ㅡ굿,NaN,굿
3758,3897,6645de8483f867cdd7e8996b,대한민국 구석구석,5,2015년 2월 25일,박서규,박서규,박 서규,박서 규
3759,3898,6645de8583f867cdd7e8996c,대한민국 구석구석,5,2013년 6월 29일,최곡,최곡,곡,최곡
3760,3899,6645de8583f867cdd7e8996d,대한민국 구석구석,5,2014년 12월 7일,영호,영호,영호,영호


In [233]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

###### user_score = 1 & user_score = 2 부정

In [234]:
# 텍스트 데이터 전처리 및 벡터화
vectorizer = TfidfVectorizer()
feature_12 = vectorizer.fit_transform(df_appreview_okt.query('user_score == 1 or user_score == 2')['review_clean_okt'])

In [235]:
# LDA 모델 훈련
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(feature_12)

LatentDirichletAllocation(n_components=5, random_state=42)

In [236]:
# 훈련된 LDA 모델로 문서별 토픽 할당
document_topics = lda.transform(feature_12)

# 각 문서의 주요 토픽 확인
for idx, topic in enumerate(document_topics):
    print(f"Document {idx} belongs to Topic {topic.argmax()}")

Document 0 belongs to Topic 4
Document 1 belongs to Topic 2
Document 2 belongs to Topic 4
Document 3 belongs to Topic 4
Document 4 belongs to Topic 3
Document 5 belongs to Topic 4
Document 6 belongs to Topic 4
Document 7 belongs to Topic 4
Document 8 belongs to Topic 0
Document 9 belongs to Topic 0
Document 10 belongs to Topic 3
Document 11 belongs to Topic 4
Document 12 belongs to Topic 4
Document 13 belongs to Topic 3
Document 14 belongs to Topic 1
Document 15 belongs to Topic 3
Document 16 belongs to Topic 4
Document 17 belongs to Topic 4
Document 18 belongs to Topic 4
Document 19 belongs to Topic 1
Document 20 belongs to Topic 1
Document 21 belongs to Topic 0
Document 22 belongs to Topic 3
Document 23 belongs to Topic 2
Document 24 belongs to Topic 0
Document 25 belongs to Topic 4
Document 26 belongs to Topic 4
Document 27 belongs to Topic 4
Document 28 belongs to Topic 2
Document 29 belongs to Topic 1
Document 30 belongs to Topic 4
Document 31 belongs to Topic 4
Document 32 belong

In [237]:
# 토픽 모델링 결과 해석
feature_names = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    # 토픽별로 상위 단어 10개 출력
    top_word_indices = topic.argsort()[:-10-1:-1]
    top_words = [feature_names[i] for i in top_word_indices if i < len(feature_names)]
    print(f"Topic {topic_idx}: {', '.join(top_words)}")

Topic 0: 위치, 검색, 정보, 삭제, 지역, 느리다, 처리, 리지, 낭비, 숙박업소
Topic 1: 도시, 불편, 종료, 일정, 연결, 광고, 오픈, 부탁, 정보, 내용
Topic 2: 접속, 테스트, 느리다, 검색, 홍보, 상태, 예약, 등록, 가족, 그대로
Topic 3: 취소, 지도, 생각, 수수료, 정보, 예약, 발생, 많다, 인터넷, 항공권
Topic 4: 예약, 연락, 최악, 해도, 전화, 문의, 음식점, 등록, 메일, 장소


###### 별점 1 & 2 점 리뷰 topic
- Topic 0: 위치, 검색, 정보, 삭제, 지역, 느리다, 처리, 리지, 낭비, 숙박업소
- Topic 1: 도시, 불편, 종료, 일정, 연결, 광고, 오픈, 부탁, 정보, 내용
- Topic 2: 접속, 테스트, 느리다, 검색, 홍보, 상태, 예약, 등록, 가족, 그대로
- Topic 3: 취소, 지도, 생각, 수수료, 정보, 예약, 발생, 많다, 인터넷, 항공권
- Topic 4: 예약, 연락, 최악, 해도, 전화, 문의, 음식점, 등록, 메일, 장소

주목할 keyword : 위치, 검색, 정보, 낭비, 숙박업소, 도시, 광고, 가족, 수수료, 지도, 항공권

###### user_score = 3 중립

In [238]:
# 텍스트 데이터 전처리 및 벡터화
vectorizer = TfidfVectorizer()
feature_33 = vectorizer.fit_transform(df_appreview_okt.query('user_score == 3')['review_clean_okt'])

In [239]:
# LDA 모델 훈련
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(feature_33)

LatentDirichletAllocation(n_components=5, random_state=42)

In [240]:
# 훈련된 LDA 모델로 문서별 토픽 할당
document_topics = lda.transform(feature_33)

# 각 문서의 주요 토픽 확인
for idx, topic in enumerate(document_topics):
    print(f"Document {idx} belongs to Topic {topic.argmax()}")

Document 0 belongs to Topic 4
Document 1 belongs to Topic 2
Document 2 belongs to Topic 2
Document 3 belongs to Topic 3
Document 4 belongs to Topic 1
Document 5 belongs to Topic 4
Document 6 belongs to Topic 2
Document 7 belongs to Topic 1
Document 8 belongs to Topic 1
Document 9 belongs to Topic 1
Document 10 belongs to Topic 0
Document 11 belongs to Topic 0
Document 12 belongs to Topic 0
Document 13 belongs to Topic 4
Document 14 belongs to Topic 0
Document 15 belongs to Topic 3
Document 16 belongs to Topic 3
Document 17 belongs to Topic 3
Document 18 belongs to Topic 2
Document 19 belongs to Topic 4
Document 20 belongs to Topic 3
Document 21 belongs to Topic 1
Document 22 belongs to Topic 2
Document 23 belongs to Topic 4
Document 24 belongs to Topic 0
Document 25 belongs to Topic 1
Document 26 belongs to Topic 1
Document 27 belongs to Topic 4
Document 28 belongs to Topic 4
Document 29 belongs to Topic 1
Document 30 belongs to Topic 4
Document 31 belongs to Topic 1
Document 32 belong

In [241]:
# 토픽 모델링 결과 해석
feature_names = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    # 토픽별로 상위 단어 10개 출력
    top_word_indices = topic.argsort()[:-10-1:-1]
    top_words = [feature_names[i] for i in top_word_indices if i < len(feature_names)]
    print(f"Topic {topic_idx}: {', '.join(top_words)}")

Topic 0: 정보, 일정, 검색, 많다, 불편, 시간, 버전, 정리, 도시, 오픈
Topic 1: 해외, 정도, 검색, 일정, 용도, 소개, 많다, 자체, 복잡, 직관
Topic 2: 느리다, 지도, 주변, 참고, 도시, 맛집, 검색, 해주시, 리지, 장소
Topic 3: 설정, 광고, 위치, 검색, 지역, 장소, 정보, 처음, 등록, 제주도
Topic 4: 일정, 저장, 장소, 어디, 정보, 메모, 좀더, 필요, 검색, 삭제


###### 별점 3 점 리뷰 topic
- Topic 0: 정보, 일정, 검색, 많다, 불편, 시간, 버전, 정리, 도시, 오픈
- Topic 1: 해외, 정도, 검색, 일정, 용도, 소개, 많다, 자체, 복잡, 직관
- Topic 2: 느리다, 지도, 주변, 참고, 도시, 맛집, 검색, 해주시, 리지, 장소
- Topic 3: 설정, 광고, 위치, 검색, 지역, 장소, 정보, 처음, 등록, 제주도
- Topic 4: 일정, 저장, 장소, 어디, 정보, 메모, 좀더, 필요, 검색, 삭제

주목할 keyword : 시간, 정리, 도시, 오픈, 지도, 차목, 리지, 광고, 메모, 정보, 검색, 일정

###### user_score = 4 & user_score = 5

In [242]:
# 텍스트 데이터 전처리 및 벡터화
vectorizer = TfidfVectorizer()
feature_45 = vectorizer.fit_transform(df_appreview_okt.query('user_score == 4 or user_score == 5')['review_clean_okt'])

In [243]:
# LDA 모델 훈련
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(feature_45)

LatentDirichletAllocation(n_components=5, random_state=42)

In [244]:
# 훈련된 LDA 모델로 문서별 토픽 할당
document_topics = lda.transform(feature_45)

# 각 문서의 주요 토픽 확인
for idx, topic in enumerate(document_topics):
    print(f"Document {idx} belongs to Topic {topic.argmax()}")

Document 0 belongs to Topic 3
Document 1 belongs to Topic 2
Document 2 belongs to Topic 2
Document 3 belongs to Topic 2
Document 4 belongs to Topic 0
Document 5 belongs to Topic 2
Document 6 belongs to Topic 4
Document 7 belongs to Topic 2
Document 8 belongs to Topic 3
Document 9 belongs to Topic 2
Document 10 belongs to Topic 4
Document 11 belongs to Topic 3
Document 12 belongs to Topic 4
Document 13 belongs to Topic 3
Document 14 belongs to Topic 3
Document 15 belongs to Topic 2
Document 16 belongs to Topic 3
Document 17 belongs to Topic 2
Document 18 belongs to Topic 2
Document 19 belongs to Topic 3
Document 20 belongs to Topic 3
Document 21 belongs to Topic 3
Document 22 belongs to Topic 2
Document 23 belongs to Topic 2
Document 24 belongs to Topic 1
Document 25 belongs to Topic 3
Document 26 belongs to Topic 3
Document 27 belongs to Topic 3
Document 28 belongs to Topic 3
Document 29 belongs to Topic 1
Document 30 belongs to Topic 2
Document 31 belongs to Topic 3
Document 32 belong

In [245]:
# 토픽 모델링 결과 해석
feature_names = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    # 토픽별로 상위 단어 10개 출력
    top_word_indices = topic.argsort()[:-10-1:-1]
    top_words = [feature_names[i] for i in top_word_indices if i < len(feature_names)]
    print(f"Topic {topic_idx}: {', '.join(top_words)}")

Topic 0: 추천, 깔끔, 괜찮다, 계획, 친구, 생각, 일정, 정보, 애용, 해외
Topic 1: 가계부, 코스, 용도, 이용, 상당, 기전, 제주도, 일정, 이제, 준비
Topic 2: 일정, 계획, 동선, 지도, 거리, 공유, 경로, 장소, 해외, 표시
Topic 3: 정보, 많다, 일정, 편리, 계획, 필요, 맛집, 다양, 추천, 정리
Topic 4: 일정, 도움, 필수, 준비, 많다, 굳다, 계획, 구석, 리뷰, 매우


###### 별점 4 & 5 점 리뷰 topic
- Topic 0: 추천, 깔끔, 괜찮다, 계획, 친구, 생각, 일정, 정보, 애용, 해외
- Topic 1: 가계부, 코스, 용도, 이용, 상당, 기전, 제주도, 일정, 이제, 준비
- Topic 2: 일정, 계획, 동선, 지도, 거리, 공유, 경로, 장소, 해외, 표시
- Topic 3: 정보, 많다, 일정, 편리, 계획, 필요, 맛집, 다양, 추천, 정리
- Topic 4: 일정, 도움, 필수, 준비, 많다, 굳다, 계획, 구석, 리뷰, 매우

주목할 keyword : 가계부, 코스, 일정, 친구, 계획, 동선, 지도, 거리, 장소, 표시, 정보, 맛집, 준비

#### 2. 긍정 point 극대화 방안

##### 긍정 review 
주목할 keyword : 가계부, 코스, 일정, 친구, 계획, 동선, 지도, 거리, 장소, 표시, 정보, 맛집, 준비

In [268]:
def keyword_contents_positive(text):
    total_lists = df_appreview.query('user_score == 4 or user_score == 5')['user_comments']
    filter = []
    for total_list in total_lists:
        try:
            if text in total_list:
                total_list = str(total_list)
                print(total_list)
                print("--------")
            else : 
                pass
        except:
            print(total_list)
            pass

In [269]:
keyword_contents_positive('가계부')

트리플 덕분에 최근에 다녀온 여행 계획을 편하게 짰습니다. 다만, 아쉬운 점이 있습니다. 1. '가계부'에 지출항목이 좀 더 다양하면 좋겠습니다. 교통비를 예를 들면 '항공', '교통' 두가지 밖에 없는데, '배', '기차', '버스' 정도의 대표적 교통 수단은 추가되었으면 합니다. 2. '배낭톡'에 검색 기능이 있으면 좋겠습니다. 나라, 도시, 여행시기, 주제별 정렬 기능은 있는데, 검색 기능이 없다보니 이용자가 원하는 여행 정보를 찾으려면 톡을 하나하나 열어서 확인해야하는 불편함이 있습니다. 고려해주세요^^
--------
진짜 편함. 특히 친구들이랑 같이 여행할 때 계획 짜고, 가계부 적어서 자동으로 1/n 하는거, 날씨나 환율 확인 바로 되는 거 최고. 하나의 아쉬운점은 오프라인에서도 일정은 볼 수 있었으면 좋겠다.
--------
너무 편하게 잘 쓰고 있어요. 다만 일행과 공유하는 가계부에서 정산이 완료된 항목을 체크하고 1/n 정산에서 제외시키는 기능이 있었으면 좋겠어요. 누가 결제했는지도 기억해두고 싶은데 정산에서 제외하려면 같이 결제한 것으로 체크해야 하니 불편합니다.
--------
여행계획 짜면서 알게된 어플이에요. 사용하기에 편한 UI와 여행에 꼭 필요한 기능이 알차게 압축되어있는 정말 좋은 앱이네요!!! 가서 직접 사용해볼 날이 기대됩니다^^ 몇가지 추가됐으면 하는 기능은 1. 여행다이어리(나혼자볼수있는) 2. 가계부기능( 환율과, 돈을 함께 쓰는 친구연동이 되는) 3. 스케쥴 짤때 시간별 작성 기능과 지도에 순서대로 표시되도록 하는 기능 이 세개만 있으면 정말 완벽할거 같아요! 따로 여행용 블로그를 하지 않아도 앱 한번에 히스토리 확인도 가능하고, 다음에 혹 다시 가게되더라도 매우 유용한 앱일거 같습니다^^ 좋은앱 감사합니다. 더발전하는 트리플되세용ㅋㅋ
--------
장점 1. 일정을 설계할 때 관광지에 대한 설명과 다양한 정보 및 사진을 볼 수 있어 여행 계획할 때 완전 좋습니다. 2. 일행과 공유할 수 있고 수정할 수 있으며 정산도

In [270]:
keyword_contents_positive('코스')

써본 여행어플 중 가장 유용한 어플인 것 같아요! 정보 검색하고 여행 계획짜는걸 정말 귀찮아하는 사람인데 추천 여행코스부터 맛집, 행사 등 유용한 정보들이 많아서 트리플 덕분에 일정짜는게 정말 간편해졌어요. 덕분에 유럽 여행 재밌게 다니고 있습니다 :) (유럽에 한해서) 아직 모든 나라, 도시들이 열린게 아니라서 아쉬운 것 빼고는 최고입니다❤
--------
여행갈때마다 유용하게 써서 리뷰 남겨요~ 여행전에 대략적 개관을 소개 해 주는 게시물 많아서 좋아요. 여행 전에 도시를 깊게 알아보기 힘든 때가 많은데 트리플 보면 내가 가는 곳이 어떤 곳인지 간단하게 알 수 있어요. 여행 코스를 짜는 것도 좋지만 UI가 편하고 직관적으로 되어 있어서 오랜만에 여행을 위해 들어가도 적응이 편해요.
--------
매번 여행갈때마다 너무잘쓰고있습니다 !! 날짜별로 여행 코스(장소)를 순서대로 적을수있다는점, 추가한 장소에서 비용을 바로 추가할수 있다는점이 이 어플에서 제게 가장 좋은 장점이었던것 같습니다! 비용 정산할때 정말 편합니다. 광고도 없어서 사용하기 좋습니다. 오래오래 어플 운영해주세요 :)
--------
12.30~ 1.2까지 첫 자유여행에 도움이 많이 되었어요. 코스를 짜기도 쉽고. 코스간의 이동시간이나 거리계산이 쉬웠어요. 자동차를 렌트해서 다녔는데 맵코드가 나와있어서 좋고. 지도로 보기도 좋고 길찾기가 쉬웠어요. 연말연초라서 영업시간이 약간 변동있었으나 영업시간은 거의 비슷하고 근처맛집과 관광지 찾기에 매우 만족햇어요! 여행중인 분들이 자세한 후기를 올리면 더욱 많은 정보가 생겨서 여행에 도움이 될거같네요. 저는 다음 여행도 트리플입니다
--------
자랑은 아니지만 스마트폰 쓴지 거의 10년짼데 수많은 어플들을 다운 받고 쓰고 하면서 리뷰 남긴거 이 어플이 처음입니다ㅋㅋㅋㅋㅋ 진짜 마음에 들어요! 특히 이번에 업데이트되서 내가 추가해놓은 장소간 거리도 나오고 그 장소들을 지도에서 보여줘서 가는길대로 코스도 짜놓을 수 있고 진짜 제 인생 최고의 어플입니다

In [271]:
keyword_contents_positive('일정')

여행 일정 잡을 때. 가장 애용하고 손이 많이 가는 앱입니다 :) 너무너무 아쉬운 점은 pc버전이 없다는 게ㅜ.. 여행일정은 결국 마지막엔 각잡고 pc로 해야 할 때가 많아서...트리플 쓰다가 결국 불편해서 구글맵을 쓰게되는..ㅜ 한눈에 확인하기에도 트리플이 짱인데~~!!!!공유도 너무 잘되구용ㅜ 그래서 pc버전이 없는 게 너무너무 아쉬워용ㅜㅅㅜ 아 편집 ->복사 기능도 있으면 좋을것같아요ㅎㅎ🥰
--------
갤럭시 폴드하고 태블릿에서 더 좋은 사용성을 가진 UX가 제공되서 좋네요 다만, PC버전과 오프라인모드 지원해 주시면 더할 나위 없을꺼 같아요 계획한 일정을 볼 때는 모바일이 편한데 일정계획 짤때는 PC에서 할때가 더 편해서; 궂이 복잡하게 말고 웹사이트로라도 제공되었으면 좋겠습니다!
--------
진짜 편함. 특히 친구들이랑 같이 여행할 때 계획 짜고, 가계부 적어서 자동으로 1/n 하는거, 날씨나 환율 확인 바로 되는 거 최고. 하나의 아쉬운점은 오프라인에서도 일정은 볼 수 있었으면 좋겠다.
--------
같이 일정을 짜고 공유할 수 있다는 게 좋고 무엇보다 여기저기 영상 뒤지고 인스타 피드 뒤지고 하지 않아도 대표 관광지 대표 스팟은 이미 정리가 잘 되어 있아서 일정짜기 편하네요. 진짜 여행고수가 만들어둔 앱인듯..!
--------
타 여행계획 어플보다 확실히 UX적으로 편안합니다. 일정공유를 할 수 있는 점과 일정을 날짜, 시간대 별 조정이 아주 쉬워서 계속 이 어플로 사용할 것 같네요. 기능 하나 추가했으면 좋겠습니다. 가고 싶은 맛집리스트를 한번에 골라둔 뒤 그 중에서 필터를 골라 (거리 가까운 순, 점심/저녁 장사 분류, 아이 동반 가능 여부 등) 일정에 끼워넣을 수 있는 방식으로요.
--------
주변 사람들에게 여행 전 꼭 추천하는 앱입니다. 동선을 보며 일정을 짤 수 있고, 여행경비 정산도 너무 편리하고 좋습니다. 몇 가지 개선할 부분은 /첫째, 2일 이상 일정의 경우에도 날짜별 색을 달리 하여 하나의 지도에 동선이 모두 표시가

In [266]:
keyword_contents('친구')

진짜 편함. 특히 친구들이랑 같이 여행할 때 계획 짜고, 가계부 적어서 자동으로 1/n 하는거, 날씨나 환율 확인 바로 되는 거 최고. 하나의 아쉬운점은 오프라인에서도 일정은 볼 수 있었으면 좋겠다.
--------
Enfp도 계획적으로 여행떠날수 있다!:) 즉흥여행을 떠나기 좋아하지만 함께가는 친구와 서로의 시간을 소중히 쓰기위하 여행일정을 위해서는 계획의 도움을 받아야 완벽해지는것을 도와준 어플! 장소가 없는 곳들도 있었지만 오히려 새로운 장소 등록조차 퀘스트미션 깨는 이 설레임은 뭐지:) 제일 깔끔하게 일정정리,공유가능한 최고의 여행어플.
--------
여행계획 짜면서 알게된 어플이에요. 사용하기에 편한 UI와 여행에 꼭 필요한 기능이 알차게 압축되어있는 정말 좋은 앱이네요!!! 가서 직접 사용해볼 날이 기대됩니다^^ 몇가지 추가됐으면 하는 기능은 1. 여행다이어리(나혼자볼수있는) 2. 가계부기능( 환율과, 돈을 함께 쓰는 친구연동이 되는) 3. 스케쥴 짤때 시간별 작성 기능과 지도에 순서대로 표시되도록 하는 기능 이 세개만 있으면 정말 완벽할거 같아요! 따로 여행용 블로그를 하지 않아도 앱 한번에 히스토리 확인도 가능하고, 다음에 혹 다시 가게되더라도 매우 유용한 앱일거 같습니다^^ 좋은앱 감사합니다. 더발전하는 트리플되세용ㅋㅋ
--------
1. 여행지에 대해 간략한 설명을 제공해줍니다. 알고 보는거랑 모르고 보는게 다르다고 생각하는 주의라 이 설명이 아주 좋았습니다. 함께 있는 입장료정보+영업시간도 좋았어요 2. 지도와 연결되는게 좋아요. 일정중에 다음 장소로 가는 길을 알 수 있어서 저 같은 길치에겐 정말 최고👍 3. 친구와 함께 일정을 공유할 수 있습니당. 동급의 UI/UX를 제공하는 어플에는 없는 기능이예요. (채팅기능도 있다면 계획 짤때 카톡을 왔다갔다 할 필요도 없을것 같아요) 4. 구글트립처럼 최적의 동선을 제공해주면 더할나위 없을 것 같아요. 구글은 딥러닝을 쓰는지는 모르겠는데, 트리플에서도 된다면 무조건 트리플만 쓸 것 같네요
--

In [283]:
keyword_contents_positive('맛집')

제주도 여행을 자주 가는데 동선 확인이용시 간단하게나마 확인 가능해서 편하게 쓰고 있어요. 다음 어플 업데이트시엔 내가 저장한 장소를 지역별로 구분해 저장할수 있으면 더 좋겠어요. 애월,한담,제주시내,서귀포,성산,구좌조천등 그 지역에 갔을때 내가 저장한 음식점이나 카페를 찾기 쉽게요.. 저장한장소가 늘다 보니 어느지역에 어디 맛집이었는지 기억이 잘 않날때가 많은데 그때 지역별 카테고리로 따로 저장해 놓은 장소만 골라 볼수 있으면 편할것 같아요, 담엔 꼭 고려해주세요~
--------
타 여행계획 어플보다 확실히 UX적으로 편안합니다. 일정공유를 할 수 있는 점과 일정을 날짜, 시간대 별 조정이 아주 쉬워서 계속 이 어플로 사용할 것 같네요. 기능 하나 추가했으면 좋겠습니다. 가고 싶은 맛집리스트를 한번에 골라둔 뒤 그 중에서 필터를 골라 (거리 가까운 순, 점심/저녁 장사 분류, 아이 동반 가능 여부 등) 일정에 끼워넣을 수 있는 방식으로요.
--------
주변 사람들에게 여행 전 꼭 추천하는 앱입니다. 동선을 보며 일정을 짤 수 있고, 여행경비 정산도 너무 편리하고 좋습니다. 몇 가지 개선할 부분은 /첫째, 2일 이상 일정의 경우에도 날짜별 색을 달리 하여 하나의 지도에 동선이 모두 표시가 되면 좋겠습니다. 전체 일정 지도를 다운받을 수 있으면 더 좋구요. /둘째, 장소추가에서 일정 주변의 숙박이나 맛집 리스트가 거리순으로 제공되면 좋겠어요. 목록 방식이나 지도방식으로요. /셋째, 여행 장소별 사진앨범도 만들어져 여행기록도 남겨서 간직할 수 있으면 좋겠어요. /좋은 어플 만들어주셔서 감사합니다. 번창하세요~
--------
써본 여행어플 중 가장 유용한 어플인 것 같아요! 정보 검색하고 여행 계획짜는걸 정말 귀찮아하는 사람인데 추천 여행코스부터 맛집, 행사 등 유용한 정보들이 많아서 트리플 덕분에 일정짜는게 정말 간편해졌어요. 덕분에 유럽 여행 재밌게 다니고 있습니다 :) (유럽에 한해서) 아직 모든 나라, 도시들이 열린게 아니라서 아쉬운 것 빼고는 최고입

#### 3. 부정 point를 통한 잠재고객의 요구사항 점검

##### 부정 review 
주목할 keyword : 위치, 검색, 정보, 낭비, 숙박업소, 도시, 광고, 가족, 수수료, 지도, 항공권

In [274]:
def keyword_contents_negative(text):
    total_lists = df_appreview.query('user_score == 1 or user_score == 2')['user_comments']
    filter = []
    for total_list in total_lists:
        try:
            if text in total_list:
                total_list = str(total_list)
                print(total_list)
                print("--------")
            else : 
                pass
        except:
            print(total_list)
            pass

In [275]:
keyword_contents_negative('위치')

괜히 설치했네. 위치 찍으면 주변 숙소정보를 알려주는 구글 보다 좋은점 일도 없음.
--------
지도에서 위치 고를때 추천해주는 장소에는 선택을 아예 못하게 되어있음 가지말라는건지 뭔...
--------
안녕하세요 트위터 광고보고 끌려서 왔네요 리뷰가 아직 미비로 보이는데...개인적 필기타블릿 필기폰을 활용하는 입장에서 길치와 방향치로써 지도도 보고 일정도 보면서 앱만 잘 만들면 괜찮은 앱 하나가 추가되겠네요^^ 추가 로그,위치카메라좌표찍기,위치메모좌표 이런것도 전 유용하던데요...
--------
스케줄 짜기엔 좋은 어플인데\n구글에서 찾은 가고 싶은 장소가 트리플에 맛집으로 등록이 안되어 있으면 스케줄에 장소를 추가 하지 못해서 불편함\n위치 검색되도록 등록하려면 신청해서 답변기다려야된다는대..\n언제 그짓거리를하라는 건지
--------
최악. 위치허용 켜도 계속 꺼지고 허용하라는 알림뜨고. 지역별 가볼만한곳은 도대체 전체 보기 기능은 왜없는지? 세금만 축내는 집단.
--------
다좋은데요~ 내위치 누르면 주위관광지 가 뜬다면 더없이 좋을것같아요 이기능이 없는 어플을 사용하다가 이기능이 있는 어플이 생긴다면 여기어플 회원상당수가 흡수될것같아요~ 타지역에 놀러갓는데 그지역의 관광지만 뜬다면 지도에서 일일이 검색해야하는 번거로움을 없앨수잇죠 이기능없는 어플기준으로 가려내는 사람들도 상당할만큼 이차이가크다고 생각들어요~~
--------
어플깔고 위치 허용 눌렀는데 할인 바코드찍어도 어플설치하라는 오류메시지만 뜸 나만 할인못받음 개짜증 직원이 했는데도 왜이러는지 모르겠다고함 제대로 만들지도 않을거면 이딴거하지마쇼
--------
위치 안맞구. 정보가 너무 없어요. 노력좀하이소
--------
위치를 켜도 내위치를 못찾는 앱
--------
위치검색 기능은 뭐하러 넣어놨나요? 안되는걸
--------
내 위치 찾기를 하면, 하세월이네요! 찾아주긴 하나요?
--------
위치정보 끄고 어플 좀 쓸 수 있게 해 주세요.
--------
참 좋은 앱인데 말

In [276]:
keyword_contents_negative('검색')

인터렉션만 화려하지 실상 일정을 짜려고 하면 지원안되는 기능이 너무 많음. 동선 최적화라든지 장소마다 설명이나 관련 링크 체크리스트를 작성할 스페이스도 없고 스팟 사이에 소요시간 계산이나 교통편 조회 기능도 없음...장소 검색쿼리도 부실...저능 다시 즐겨 사용하던 어스토리로 넘어갑니다....위 기능들 보완하면 다시 돌아올게요 기획팀 열일해주세요
--------
앱 내의 지도는 한 10년도 더 된 지도라서 쓸모가 없으며 따라서 필연적으로 나만의 장소를 등록해서 일정에 추가해야 하지만 인터페이스가 너무 구려서 일정에서 나만의 장소를 추가할 수가 없다. 심지어 KTX 정차역인 용산역이 검색이 안 된다. 옆에서 하는 말이 "깔아는 놨는데 사용하지 않게 되더라." 동감..
--------
앱 지도에는 뜨는데 왜 검색해서 추가하려면 없다고 하나요? 지도 하나하나 움직여서 찾아 클릭해도 안되고
--------
쓰레기 어플입니다. 지도검색도 제대로 안 되는건 물론이고, 쓰기도 불편합니다. 그냥 네이버 지도로 검색하고 메모장에 남겨서 다니는게 훨 편합니다. 공유 되는 기능이 있지만 지도 검색기능이 약해서 쓰느니 못합니다. 기획하신 분은 그냥 개발 관두시고 한적한 곳에서 농사나 지으세요
--------
장소 추가를 맵에서 바로 찍어서 할 수 있으면 좋겠어요. 아니면 주소를 확인할 수 있게라도 한다든지... 검색이 안되는 가게도 많고 장소 추가하려면 일부러 구글맵등으로 정확한 주소를 검색해야하는 게 번거로운데 굳이 그렇게까지 할 분들이 많을지는 모르겠네요.
--------
좋다고해서 써보려고 햇더니, 타이중 검색안되서 못쓰고.. 다음여행으로 미국 올랜도도 검색안되서 못씀ㅋㅋㅋ 좋은거 맞는지 모르겟넼ㅋㅋ 차라리 관광지 선택 안되더라도 지역만 선택하게 해주던가 ㅋㅋ
--------
오사카 전용이네요 외부지역은 아예 검색도 안되고 직접 주소 쳐야하고 하다 못해 우동으로 유명한 다카마쓰는 나오지도 않음 가장 유명한 리쓰린 공원도 안나오고 오사카역치면 뭔 식당만나옴 우메다쳐야하고 하나하

In [281]:
keyword_contents_negative('숙박업소')

숙박업소와 소통이 잘 안 되고 일 처리도 원만하지 않은 것 같아요. 몇 천원 할인 받으려다가 몇 만원 날렸네요. 불가피한 사정이 생겨서 취소나 연기를 협의해 달라고 요청했는데 숙박업소와 연락이 안 된다고 했습니다. 믿을 수가 없었어요. 차라리 숙박업소의 취소환불 정책상 불가능하다고 했으면 처음부터 포기했을 텐데 너무 아쉬웠습니다.
--------
여행지를 선택하면 관광지가 우선으로 나오는게 아니라 숙박업소가 먼저 나오네요 숙박업소 카테고리를 따로 분류했으면 합니다 여행은 숙소를 정하고 관광지를 정하기보다 관광지를 정하고 숙소는 그 다음입니다 댓글답변 관광지 태그가 있긴하죠 그러면 관광지가 처음에 나오고 숙박이 태그 선택하면 나오게도 가능한거잖아요 숙박앱도 아니고 여행앱이 숙박이 우선이라는건 숙박업소 홍보로 보이며 태그 선택 모르는 사람들은 숙박업소가 관광지보다 먼저 나오면 보기 안좋을것 같습니다
--------
음식점&숙박업소 광고용 인듯
--------


In [279]:
keyword_contents_negative('광고')

다른 여행일정 어플처럼 장소추가가 가능한줄 알았는데 문의결과 그런 기능이 없음. 무조건 트리플에서 제공되는 정보로만 여행계획을 짤 수 있어 여행사는 아니지만 여행사같은 어플과 비슷함. 여행에미치다에서 소개가 나왔었는데 광고인 것 같음. 사용자 참여 개선이 필요함.
--------
내 리뷰 왜 지워졌지??? 국민수학여행지 수준인 도시 아니면 오픈이 안 되어있어서 쓸 데가 없다. 대체로 구글맵이 더 유용함. 광고 엄청 해대길래 깔아봤는데 별로예요.
--------
인스타 광고는 무슨 여행일정짜는거랑 여행장소 추천해주는것 처럼 해놓고 들어오니까 별로 싸지도않은 숙소추천만 하는 어플이네
--------
최악 있는곳만 있고 별로임 광고하고 그러는 앱같음. 유투브 광고하고 최악임
--------
안녕하세요 트위터 광고보고 끌려서 왔네요 리뷰가 아직 미비로 보이는데...개인적 필기타블릿 필기폰을 활용하는 입장에서 길치와 방향치로써 지도도 보고 일정도 보면서 앱만 잘 만들면 괜찮은 앱 하나가 추가되겠네요^^ 추가 로그,위치카메라좌표찍기,위치메모좌표 이런것도 전 유용하던데요...
--------
음식점&숙박업소 광고용 인듯
--------


In [282]:
keyword_contents_negative('가족')

트리플아. 트리플로 해외 호텔예약하고 현지 도착했는데, 하나투어가 예약취소했다고 하네? 이게 무슨 개떡같은 상황입니까. 이 오밤중에. 서비스를 감당하지 못하면 하지 마십시오. 어렵게 연차쓰고 가족 여행왔는데 지금 여행 다 망했습니다.
--------
앱을 실행시켜도 처음 화면에서 넘어가지지가 않네요 제것만 그런게 아니라 가족것도 똑같네요.. 뭔 이런 어플이 있는지..
--------
가족 여행때문에 설치했습니다. 관광지가 지역별로 리스트는 잘 되있으나 관광지 클릭해서 정보 확인 후 뒤로 가기를 누르면 보고있던 지역 리스트 목록이 안나오고 초기화 되서 다시 첨부터 들어가야하는 점이 불편하네요.
--------
앱을 만들어놓고선 뭐 되는게 없네요.. 앱 깔아서 뭐 정보 얻고 할인받으시려는분들 그냥 깔지 마시기 바랍니다. 계속 튕기고.. 진행도 안되고.. 가족 전체가 다 그래서 여행지에서 기분 망쳤습니다.
--------
